
# Amazon Reviews Scraping

In [1]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# Header to set the requests as a browser requests
'''
These headers are used to mimic a browser request. This is a common 
practice in web scraping to prevent websites from blocking your requests.
'''
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [14]:
# url of The amazon Review page
reviews_url = 'https://www.amazon.in/Apple-iPhone-14-128GB-Blue/product-reviews/B0BDK62PDX/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

In [15]:
# number of pages to scrape.
len_page = 4

### Functions

In [16]:
# This function retrieves the HTML data from multiple pages of 
# Amazon reviews.

# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):
    
    # Empty List define to store all pages html data
    soups = []
    
    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):
        
        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        # Request make for each page
        response = requests.get(url, headers=headers)
        
        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Add single Html page data in master soups list
        soups.append(soup)
        
    return soups

In [17]:
# This function extracts relevant information from the HTML data of each review.
# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []
    
    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')
    
    # Iterate all Reviews BOX 
    for box in boxes:
        
        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data 
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)
    
    return data_dicts

### Data Process

In [18]:
# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

In [19]:
# Empty List to Hold all reviews data
reviews = []

In [20]:
# Iterate all Html page 
for html_data in html_datas:
    
    # Grab review data
    review = getReviews(html_data)
    
    # add review data in reviews empty list
    reviews += review

In [21]:
# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [22]:
df_reviews

,Name,Stars,Title,Date,Description
0,Prasanna,5.0,5.0 out of 5 stars\nExceeding my expectation. ...,N/A,About the phoneApple phone without a second de...
1,Vaishnavi Nelabotla,5.0,5.0 out of 5 stars\nLoved it ❤️,N/A,Buying an iphone with my first salary is the d...
2,Jayakrishnan,4.0,4.0 out of 5 stars\nIt's visually appealing an...,N/A,Pros: Good Looking (4/5): The Blue color vari...
3,Marvellous,5.0,5.0 out of 5 stars\nIphone 14,N/A,Good product
4,Amazon Customer,5.0,5.0 out of 5 stars\nExcellent product,N/A,Excellent product
5,Hariharan Balakrishnan,5.0,5.0 out of 5 stars\nWorthy gift for better half,N/A,Bought the 256gb iPhone product RED. The color...
6,Phone is absolutely perfect,5.0,5.0 out of 5 stars\nBe care full when package ...,N/A,I like the product very much
7,Unique,4.0,4.0 out of 5 stars\nPerformance is good but ba...,N/A,I am using it from 18 January 2024 only in 2 a...
8,Akshat Sharma,4.0,4.0 out of 5 stars\nAll that glitters is not gold,N/A,It’s a good phone but don’t think that it’s pe...
9,MWO SK SINGH,4.0,4.0 out of 5 stars\nReview,N/A,Security features


In [13]:
# Save data
df_reviews.to_csv('reviews.csv', index=False)